In [9]:
# 1. Importación de Librerías
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.preprocessing import LabelEncoder
import torch
import json  # <-- Importamos la librería para trabajar con JSON

import pandas as pd

In [2]:
# 2. Declaración de Funciones

def predict_sentiment(text, model, tokenizer, label_encoder, device):
    model.eval()
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {key: value.to(device) for key, value in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    predicted_class = torch.argmax(outputs.logits, dim=1)
    return label_encoder.inverse_transform(predicted_class.cpu())[0]

In [3]:
# 3. Inicialización del Modelo

tokenizer = BertTokenizer.from_pretrained("ignacio-ave/BETO-nlp-sentiment-analysis-spanish")
model = BertForSequenceClassification.from_pretrained("ignacio-ave/BETO-nlp-sentiment-analysis-spanish")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

label_encoder = LabelEncoder().fit(['P+', 'P', 'NEU', 'NONE', 'N', 'N+'])

In [5]:

my_tweet = "Don Juan Fuentes ayer salió de paseo por las nuevas estaciones de #L3. “El Metro es lo más bacán que ha pasado en Quilicura” 🤎 Gracias don Juan, para nosotros es un honor 🚇"

prediction = predict_sentiment(my_tweet, model, tokenizer, label_encoder, device)
result_individual = {
    "Tweet": my_tweet,
    "Predicción": prediction
}

print(json.dumps(result_individual, ensure_ascii=False, indent=4))  # <-- Imprimimos en formato JSON

{
    "Tweet": "Don Juan Fuentes ayer salió de paseo por las nuevas estaciones de #L3. “El Metro es lo más bacán que ha pasado en Quilicura” 🤎 Gracias don Juan, para nosotros es un honor 🚇",
    "Predicción": "P+"
}


In [6]:
## Multiple

tweets = [
    "Me encanta usar este modelo para análisis de sentimiento.",
    "Este producto no cumplió mis expectativas.",
    "El clima está bastante agradable hoy.",
    "Me siento devastado por las noticias recientes.",
    "La película estuvo regular, no fue ni buena ni mala."
]

In [7]:
results_multiple = []

for tweet in tweets:
    prediction = predict_sentiment(tweet, model, tokenizer, label_encoder, device)
    results_multiple.append({
        "Tweet": tweet,
        "Predicción": prediction
    })

print(json.dumps(results_multiple, ensure_ascii=False, indent=4))  # <-- Imprimimos en formato JSON


[
    {
        "Tweet": "Me encanta usar este modelo para análisis de sentimiento.",
        "Predicción": "P+"
    },
    {
        "Tweet": "Este producto no cumplió mis expectativas.",
        "Predicción": "N"
    },
    {
        "Tweet": "El clima está bastante agradable hoy.",
        "Predicción": "P+"
    },
    {
        "Tweet": "Me siento devastado por las noticias recientes.",
        "Predicción": "N+"
    },
    {
        "Tweet": "La película estuvo regular, no fue ni buena ni mala.",
        "Predicción": "P"
    }
]


In [14]:

def predict_sentiment(text, model, tokenizer, label_encoder, device):
    model.eval()
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {key: value.to(device) for key, value in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    # Convertir logits a probabilidades usando softmax
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=1)
    return probabilities[0].cpu().numpy()

labels = ['P+', 'P', 'NEU', 'NONE', 'N', 'N+']
label_encoder = LabelEncoder().fit(labels)

In [15]:
# 4. Realizar una Predicción

## Individual

my_tweet = "Don Juan Fuentes ayer salió de paseo por las nuevas estaciones de #L3. “El Metro es lo más bacán que ha pasado en Quilicura” 🤎 Gracias don Juan, para nosotros es un honor 🚇"
probabilities = predict_sentiment(my_tweet, model, tokenizer, label_encoder, device)

print(f"Tweet: {my_tweet}")
print("---")
for label, probability in zip(labels, probabilities):
    print(f"{label}: {probability:.3f}")
print("\n")

Tweet: Don Juan Fuentes ayer salió de paseo por las nuevas estaciones de #L3. “El Metro es lo más bacán que ha pasado en Quilicura” 🤎 Gracias don Juan, para nosotros es un honor 🚇
---
P+: 0.000
P: 0.001
NEU: 0.001
NONE: 0.000
N: 0.005
N+: 0.992




In [16]:
## Multiple

tweets = [
    "Me encanta usar este modelo para análisis de sentimiento.",
    "Este producto no cumplió mis expectativas.",
    "El clima está bastante agradable hoy.",
    "Me siento devastado por las noticias recientes.",
    "La película estuvo regular, no fue ni buena ni mala."
]

for tweet in tweets:
    probabilities = predict_sentiment(tweet, model, tokenizer, label_encoder, device)
    print(f"Tweet: {tweet}")
    print("---")
    for label, probability in zip(labels, probabilities):
        print(f"{label}: {probability:.3f}")
    print("\n")

Tweet: Me encanta usar este modelo para análisis de sentimiento.
---
P+: 0.000
P: 0.001
NEU: 0.001
NONE: 0.001
N: 0.021
N+: 0.976


Tweet: Este producto no cumplió mis expectativas.
---
P+: 0.906
P: 0.062
NEU: 0.025
NONE: 0.003
N: 0.002
N+: 0.001


Tweet: El clima está bastante agradable hoy.
---
P+: 0.000
P: 0.001
NEU: 0.001
NONE: 0.001
N: 0.013
N+: 0.985


Tweet: Me siento devastado por las noticias recientes.
---
P+: 0.471
P: 0.512
NEU: 0.013
NONE: 0.001
N: 0.001
N+: 0.001


Tweet: La película estuvo regular, no fue ni buena ni mala.
---
P+: 0.002
P: 0.001
NEU: 0.024
NONE: 0.001
N: 0.613
N+: 0.360


